Análise: distribuição dos modelos de urna por local de votação.
--------------------------------------------------------------

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Extraído do arquivo modelourna_numerointerno.csv disponibilizado pelo TSE
MODELOS = {
    # ano:  [inicial, final]
    "2009": [ 999500,  1220500 ],
    "2010": [ 1220501, 1345500 ],
    "2011": [ 1368501, 1650000 ],
    "2013": [ 1650001, 1701000 ],
    "2015": [ 1750000, 1950000 ],
    "2020": [ 2000000, 2250000 ],
}

ESTADOS = [
    "RS","SC","PR",
    "SP","RJ","MG","ES",
    "MS","MT","GO","DF",
    "BA","SE","AL","PE","PB","RN","CE","PI","MA",
    "TO","PA","RO","AC","AM","AP","RR",
    "ZZ"
]

COLUNAS = [
    "SG_UF","NR_ZONA","NR_SECAO","DS_CARGO_PERGUNTA","NR_LOCAL_VOTACAO","NR_URNA_EFETIVADA","NM_VOTAVEL","QT_APTOS","QT_VOTOS",
]

def modelo_by_urna(urna):
    for modelo, (inicial, final)  in MODELOS.items():
        if inicial <= urna <= final:
            return modelo
    return None


Realizamos a leitura dos arquivos estado-por-estado, removendo as linhas que
não interessam (i.e. eleição pra Governador), e adicionando uma coluna com
o modelo da urna utilizado.

In [ ]:
all_dfs = []

for estado in ESTADOS:
    nome_arquivo = f"data/original/bweb_2t_{estado}_311020221535.csv"
    arquivo = pd.read_csv(nome_arquivo, sep=";", encoding="iso-8859-1")
    dfe = pd.DataFrame(arquivo, columns=COLUNAS)
    dfe = dfe[dfe.DS_CARGO_PERGUNTA == "Presidente"]
    dfe = dfe.assign(SY_MODELO_URNA=dfe.NR_URNA_EFETIVADA.apply(modelo_by_urna))
    dfe = dfe.assign(SY_COUNT=1)
    dfe = dfe.assign(SY_LOCAL_VOTACAO_CHAVE=dfe.SG_UF + "-" + dfe.NR_ZONA.astype(str) + "-" + dfe.NR_LOCAL_VOTACAO.astype(str))
    all_dfs.append(dfe)

df = pd.concat(all_dfs)

df

Agora, pivotamos os dados, usando como chave a urna e cada candidato possível como uma
nova coluna. Isso permitirá outras análises de forma simplificada, mas a rigor não era
necessário para analisar os modelos de urna por local de votação

In [ ]:
# cs = colapsado por seção

cs = df.pivot_table(
    columns="NM_VOTAVEL",
    values="QT_VOTOS",
    aggfunc=np.sum,
    fill_value=0,
    index=[
        df.SG_UF, df.NR_ZONA, df.NR_SECAO, df.SY_MODELO_URNA,
        df.NR_LOCAL_VOTACAO, df.NR_URNA_EFETIVADA, df.SY_LOCAL_VOTACAO_CHAVE], 
).reset_index().rename_axis(None, axis=1)

cs.to_csv("secao_colapsado.csv")

cs


Aqui pivotamos novamente os dados, agora usando o _local de votação_ como chave, e o
número de urnas de cada modelo naquele local de votação como colunas. O "local de
votação", na terminologia usada pelo TSE, é cada endereço distinto que pode ser
informado para os eleitores quando estes consultam o local de votação. Tipicamente,
é um colégio ou algum outro local de grande porte.

Também adicionamos uma nova coluna contando o _número de modelos distintos_ que foi
utilizado em cada local de votação.

Por exemplo, se um dado local de votação tinha 25 urnas, das quais 20 eram modelo
2015 e 5 eram modelo 2013, a linha desse local de votação conterá:

```
UF  ZONA   LOCAL  2009  2010  2011  2013  2015  2020  SY_MODELOS_DISTINTOS
UU   12     345     0     0     0    5     20     0            2
```



In [ ]:
# mulv = modelo urna x local de votação

mulv = df.pivot_table(
    columns="SY_MODELO_URNA",
    values="SY_COUNT",
    aggfunc=np.sum,
    fill_value=0,
    index=[df.SG_UF, df.NR_ZONA, df.NR_LOCAL_VOTACAO, df.SY_LOCAL_VOTACAO_CHAVE]
)
mulv["SY_MODELOS_DISTINTOS"] = mulv.astype(bool).sum(axis=1)
mulv = mulv.reset_index().rename_axis(None, axis=1)
mulv.to_csv("urnas_por_local_votacao.csv")

mulv

Por fim, criamos um conjunto de gráficos de histograma, um para cada modelo de urna,
indicando a distribuição do _número de modelos de urnas distintas_ em locais de votação
onde pelo menos uma urna do modelo esteve presente.

Em outras palavras, se um determinado local de votação tinha apenas urnas do modelo 2010,
2011 e 2013, este local de votação estará "contribuindo" na barra `3` nos gráficos 2010, 2011
e 2013 abaixo.

In [ ]:


fig, axs = plt.subplots(2,3)
plt.rcParams['axes.titlepad'] = -20
plt.rcParams['axes.titley'] = 1
plt.subplots_adjust(bottom=0.1, right=2, top=0.9)

for index, modelo in enumerate(MODELOS):
    ax = axs[index // 3, index % 3]
    counts, bins, patches = ax.hist(mulv[mulv[modelo] > 0].SY_MODELOS_DISTINTOS, bins=5, width=0.6)
    maximum = max(*counts)
    for bin, count in zip(bins, counts):
        y = count + maximum * 0.05 if count < maximum * 0.95 else count - maximum * 0.1
        ax.text(bin + 0.1, y, str(int(count)), fontsize=6, bbox=dict(boxstyle="square,pad=0.2", fc="lightgray", ec="none"))
    ax.grid(False)
    ax.set_title(modelo, loc="right")


Com estes gráficos, podemos observar uma coisa interessante: as urnas modelo 2020
_muito raramente_ foram "misturadas" com outras urnas em um mesmo local de votação.

Isso por si só explica a aparente "distorção" onde essa urna estaria com resultados
radicalmente diferentes pois ela seria "infraudável": a distribuição das urnas
_não foi uniforme_. Ora, se a distribuição não é uniforme, não cabe aplicar testes
estatísticos cuja conclusão está lastreada na uniformidade.

In [ ]:
# lhu2020 = locais heterogeneos com urna 2020
lhu2020 = mulv[mulv["2020"] > 0][mulv.SY_MODELOS_DISTINTOS > 1]
lhu2020 = lhu2020.SY_LOCAL_VOTACAO_CHAVE.values

fig, axs = plt.subplots(2,3, sharex=True, sharey=True)
plt.rcParams['axes.titlepad'] = -20
plt.rcParams['axes.titley'] = 1
plt.subplots_adjust(bottom=0.1, right=2, top=0.9)

for index, modelo in enumerate(MODELOS):
    ax = axs[index // 3, index % 3]
    data = cs.loc[cs.SY_LOCAL_VOTACAO_CHAVE.isin(lhu2020)][cs.SY_MODELO_URNA == modelo]
    ax.scatter(data["LULA"], data["JAIR BOLSONARO"])
    ax.grid(False)
    ax.set_title(modelo, loc="right")

